## Import Libraries

In [1]:
from coinbase.wallet.client import Client
import cbpro

from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


## Connect to API for my Account

In [2]:
import config
client = Client(config.apiKey, config.apiSecret, api_version = config.apiVersion)

btcCurrencyPair = 'ETH-USD'

price = client.get_spot_price(currency_pair = btcCurrencyPair)
price

<APIObject @ 0x1c26975ea90> {
  "amount": "1432.09",
  "base": "ETH",
  "currency": "USD"
}

## Useful Functions

In [3]:
def UnixToDate(time): 
    return datetime.fromtimestamp(time)

In [4]:
def isGranularityOption(g):
    #1 min, 5 min, 15 min, 1 hr, 6 hrs, 1 day
    possibleGranularities = [60, 300, 900, 3600, 21600, 86400] 
    return (g in possibleGranularities)

In [5]:
from datetime import datetime
def unixToISO8601(unixTime):
    t =  int(unixTime)
    return str(datetime.fromtimestamp(t).isoformat())

In [14]:
def x_YearsAgoInUnixTime(years):
    today = int(time.time())
    secodnsPerYear = 31536000
    return (today - (years*secodnsPerYear))
def x_MonthsAgoInUnixTime(months):
    today = int(time.time())
    secodnsPerMonth = 2628000 #average sec/month
    return (today - (months*secodnsPerMonth))
def x_DaysAgoInUnixTime(days):
    today = int(time.time())
    secodnsPerDay = 86400
    return (today - (days*secodnsPerDay))

In [6]:
#API will only return 300 bins at once so if you input a big range you will need to call API multiple times
def calcNumberofCallsNeeded(startTime_unix, endTime_unix, granularity):
    maxBins = 300
    maxTime = maxBins * granularity
    if maxTime < endTime_unix - startTime_unix:
        return ((endTime_unix - startTime_unix)//maxTime) + 1 #add 1 cause we want ceiling(x)
    else:
        return 1

In [7]:
#takes ticker and granularity and a start and end time and returns a dataFrame of the data
import time
def getData(tickerPair, granularity, startTime_unix, endTime_unix = int(time.time())):
    if isGranularityOption(granularity) and endTime_unix > startTime_unix:  
        numCalls = calcNumberofCallsNeeded(startTime_unix,endTime_unix, granularity)

        data = []
        isoStart = unixToISO8601(startTime_unix)
        isoEnd = unixToISO8601(endTime_unix)
        if numCalls == 1:
            data = public_client.get_product_historic_rates(tickerPair, isoStart, isoEnd, granularity)
        else:
            for i in range(numCalls + 1):
                currentStep = 300*i*granularity #300 is max bins coinbase API will return at once
                isoStart = unixToISO8601(endTime_unix-currentStep)
                temp = public_client.get_product_historic_rates(tickerPair, isoStart, isoEnd, granularity)
                isoEnd = isoStart
                for element in temp: #data is 2D so loop to append, make prettier later
                    data.append(element)
           
        df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close','volume'])
        df.set_index('date', inplace=True)
        return df.iloc[::-1] #reverse order so its chronological
    else:
        print("Input error, use one of the following granularity values [60, 300, 900, 3600, 21600, 86400]\
        and make sure the start time is before the end time")
        return None

In [8]:
import plotly.graph_objects as go
def plotCandles(df):
    
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'])])
    fig.show()

In [16]:
def plotLine(df,ticker):
    plt.figure(1,figsize=(8, 5))
    plt.plot(df['close'],label = ticker)
    plt.xlabel("Unix Time")
    plt.ylabel("Price-USD")
    start = UnixToDate(df.index[-1])
    end = UnixToDate(df.index[0])
    plt.title("{}-USD from {} to {}".format(ticker,start,end))
    plt.legend()
    plt.show()

## Trade Indicator Functions

In [65]:
#given a dataframe it will calculate the RSI for each candle
def relativeStrengthIndex(df, granularity = 86400, lookbackPeriod = 14):

    data = df.drop('volume',axis = 1)
    data["change"] = np.nan
    data["RSI"] = np.nan
    data['change'] = data['close'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
    data['RSI'] = data['change'].rolling(window=lookbackPeriod).apply(relativeStrengthIndexHelper)
    
    return data['RSI'] #subject to change not sure how I'm going to use, maybe just edit initial df and dont delete volume

def relativeStrengthIndexHelperSMA(period): #period is last n changes in closing price
    avgOfPositives = np.mean(period[period > 0])
    avgOfNegatives = np.mean(period[period < 0])
    relativeStrength =  avgOfPositives/avgOfNegatives
    return (100 - (100/(1 + relativeStrength)))

def relativeStrengthIndexHelperEMA(period):
    avgOfPositives = np.mean(period[period > 0])
    avgOfNegatives = np.mean(period[period < 0])
    relativeStrength =  avgOfPositives/avgOfNegatives
    return (100 - (100/(1 + relativeStrength)))

def relativeStrengthIndexHelperSMMA(period):
    avgOfPositives = np.mean(period[period > 0])
    avgOfNegatives = np.mean(period[period < 0])
    relativeStrength =  avgOfPositives/avgOfNegatives
    return (100 - (100/(1 + relativeStrength)))

## Example -- BTC Data

In [66]:
public_client = cbpro.PublicClient()

#careful, can overflow memory and break if you get too much data
btc = getData("BTC-USD", 86400, x_YearsAgoInUnixTime(5)) #BTC data from 5+ years ago at a bin size of 1 day
#plotLine(btc,"BTC")

temp = relativeStrengthIndex(btc, granularity = 86400, lookbackPeriod = 14)
temp

date
1437350400           NaN
1437436800           NaN
1437523200           NaN
1437609600           NaN
1437696000           NaN
                 ...    
1614038400    269.497708
1614124800    319.469416
1614211200    516.833750
1614297600    540.830908
1614384000    535.527186
Name: RSI, Length: 2050, dtype: float64